# Chapter 01 - Introduction

## Fundamentals Concepts to understand the novelty around transformers

To understand what transformers are, we first need to understand the novelties behind it.

1. The encode-decoder framework

The encoder-decoder framework is based on the RNN architecture developed for multilingual translation. The encoder block receives a text sentence (in the first language, e.g., english) and encode it to numerical format into what is called "hidden state". The hidden state is then passed to the decoder block, the decodes it into the target language (e.g., german).

2. Attention Mechanisms

Due to the inhererent sequential natural of the RNNs, for long sequences, after feeding the whole sequence to the RNN, the resulting hidden state may lose the initial context of the sentence. To overcome this issue, "attention" mechanisms were invented. The attention mechanisms idea is that instead of passing only the final hidden state to the decoder, we should pass all of them, from the first to the last. This enable the model to learn non-trivial relationship between words and to overcome the memory issue of forgetting initial information. 

Lastly, one last challenge remained: the inherent sequential nature of RNN. To mitigate that problem and training models faster, the transformer architecture was introduced with the idea of "self-attention". Self-attention basically inputs words to Feed-Forward Neural Networks (FF NN), to generate and to decode the hidden states of the transformer network.

3. Transfer Learning

Transfer Learning in NLP is powered by the ULMFiT framework. Due to the lack of labeled data for NLP tasks, researchers from OpenAI proposed the ULMFiT framework that is basically three steps:

- Pretraining: Get a large corpus of data such as wikipedia, and train a model to predict the next word based on the previous one. This task is referred to as "language modeling".
- Domain adaptation: Still uses language modeling but over texts from the domain you want to work with (fine-tuning).
- Fine-Tuning: now that the model has been fine-tuned, we can put a classification head over it (or whatever we need), and perform the target task

In [1]:
import logging
import warnings

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

from transformers import pipeline

## Hugging Face Transformers

### Text Classification

In [3]:
classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [4]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


### Named Entity Recognition

In [5]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [6]:
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556569,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498350,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


### Question Answering

In [7]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)

pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


### Summarization

In [8]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)

No model was supplied, defaulted to google-t5/t5-small and revision d769bba (https://huggingface.co/google-t5/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
I0000 00:00:1716894334.255489   30501 service.cc:145] XLA service 0x7fb65f1be850 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716894334.255541   30501 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-05-28 08:05:34.255860: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performanc

In [11]:
outputs[0]["summary_text"]

'last week, I ordered an Optimus Prime action figure from your online store in germany. when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead'

### Translation

In [12]:
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")

outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-de.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

In [15]:
outputs[0]["translation_text"]

'Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur bestellte ich. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee.'

### Text Generation

In [18]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [19]:
print(outputs[0]['generated_text'])

Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. It was from your online store, not

from another supplier. I do not know where I got your code, or if it is an e-mail address or fax machine (it was

or was received by your company). Also, I have been sent with what looks like
